**Общая информация**

**Срок сдачи:** 16 марта 2020, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
from sklearn.datasets import fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [2]:
from scipy.stats import mode
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import pairwise_distances

class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors, algorithm='brute', metric='euclidean'):
        self.n_neighbors = n_neighbors
        
        if algorithm != "brute" and algorithm != "kd_tree":
            print("Error while creating class! I don't know this algorithm! Try another one!")
            exit(1)
            
        self.algorithm = algorithm
        self.metric = metric
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        
        if self.algorithm == "kd_tree":
            self.kdtree = KDTree(self.X_train)
    
    def predict(self, X):
        if self.algorithm == "brute":        
            distances = pairwise_distances(X, self.X_train, self.metric)
            k_nearest_inds = np.argpartition(distances, self.n_neighbors, axis = 1)[:, :self.n_neighbors]
            y_train_nearest = self.y_train[k_nearest_inds]
            y_pred = mode(y_train_nearest, axis = 1)[0]
            
        if self.algorithm == "kd_tree":
            nearest_dist, nearest_inds = self.kdtree.query(X, k=self.n_neighbors)
            y_train_nearest = self.y_train[nearest_inds]
            y_pred = mode(y_train_nearest, axis = 1)[0]
        
        return y_pred.reshape(-1)

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)

In [8]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [9]:
%time clf.fit(X_train, y_train)

CPU times: user 1.16 ms, sys: 262 µs, total: 1.43 ms
Wall time: 1.05 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [10]:
%time my_clf.fit(X_train, y_train)

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.3 µs


In [11]:
%time clf.predict(X_test)

CPU times: user 209 µs, sys: 4.19 ms, total: 4.4 ms
Wall time: 4.02 ms


array([0, 1, 2, 1, 0, 0, 2, 1, 1, 2, 0, 1, 2, 0, 2])

In [12]:
%time my_clf.predict(X_test)

CPU times: user 1.34 ms, sys: 303 µs, total: 1.64 ms
Wall time: 1.29 ms


array([0, 1, 2, 1, 0, 0, 2, 1, 1, 2, 0, 1, 2, 0, 2])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [13]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [15]:
%time clf.fit(X_train, y_train)

CPU times: user 1.76 ms, sys: 396 µs, total: 2.15 ms
Wall time: 23.6 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [16]:
%time my_clf.fit(X_train, y_train)

CPU times: user 165 µs, sys: 37 µs, total: 202 µs
Wall time: 180 µs


In [17]:
%time clf.predict(X_test)

CPU times: user 4.88 ms, sys: 170 µs, total: 5.05 ms
Wall time: 3.94 ms


array([2, 2, 2, 0, 2, 0, 0, 2, 1, 0, 0, 1, 2, 1, 1])

In [18]:
%time my_clf.predict(X_test)

CPU times: user 2.04 ms, sys: 457 µs, total: 2.5 ms
Wall time: 1.95 ms


array([2, 2, 2, 0, 2, 0, 0, 2, 1, 0, 0, 1, 2, 1, 1])

In [19]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [20]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [21]:
data = newsgroups['data']
target = newsgroups['target']

Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cntvctr = CountVectorizer(max_features = 10000)
data_vect = cntvctr.fit_transform(data)
data_vect

<11314x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 924130 stored elements in Compressed Sparse Row format>

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [23]:
from sklearn.model_selection import KFold

In [24]:
kf = KFold(n_splits=3)
mclsf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')
accuracy_mean = 0

for train_index, val_index in kf.split(data_vect):
    data_vect_train, data_vect_val = data_vect[train_index], data_vect[val_index]
    target_train, target_val = target[train_index], target[val_index]
    mclsf.fit(data_vect_train, target_train)
    pred = mclsf.predict(data_vect_val)
    acc = accuracy_score(pred, target_val)
    accuracy_mean += acc
    print("Accuracy score is %.4f." % acc)
    
accuracy_mean /= 3
print("\nMean accuracy on 3 folds is %.4f." % accuracy_mean)

Accuracy score is 0.1824.
Accuracy score is 0.1848.
Accuracy score is 0.1851.

Mean accuracy on 3 folds is 0.1841.


Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [25]:
def get_mean_acc(n_neighbors):
    kf = KFold(n_splits=3)
    mclsf = MyKNeighborsClassifier(n_neighbors=n_neighbors, algorithm='brute')
    accuracy_mean = 0

    for train_index, val_index in kf.split(data_vect):
        data_vect_train, data_vect_val = data_vect[train_index], data_vect[val_index]
        target_train, target_val = target[train_index], target[val_index]
        mclsf.fit(data_vect_train, target_train)
        pred = mclsf.predict(data_vect_val)
        acc = accuracy_score(pred, target_val)
        accuracy_mean += acc
    
    accuracy_mean /= 3
    return accuracy_mean

def get_best_n_neighbors():
    means = []
    for i in range(1, 11):
        acc_mean = get_mean_acc(i)
        print("Mean accuracy for %i neighbors is %.4f." % (i, acc_mean))
        means.append(acc_mean)
    return np.argmax(means) + 1

best_number = get_best_n_neighbors()
print("\nBest number of neighbors is %i." % best_number)

Mean accuracy for 1 neighbors is 0.2047.
Mean accuracy for 2 neighbors is 0.1841.
Mean accuracy for 3 neighbors is 0.1843.
Mean accuracy for 4 neighbors is 0.1815.
Mean accuracy for 5 neighbors is 0.1853.
Mean accuracy for 6 neighbors is 0.1837.
Mean accuracy for 7 neighbors is 0.1853.
Mean accuracy for 8 neighbors is 0.1903.
Mean accuracy for 9 neighbors is 0.1915.
Mean accuracy for 10 neighbors is 0.1930.

Best number of neighbors is 1.


Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

In [26]:
# сначала попробуем использовать только косинусную метрику

kf = KFold(n_splits=3)
mclsf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute', metric='cosine')
accuracy_mean = 0

for train_index, val_index in kf.split(data_vect):
    data_vect_train, data_vect_val = data_vect[train_index], data_vect[val_index]
    target_train, target_val = target[train_index], target[val_index]
    mclsf.fit(data_vect_train, target_train)
    pred = mclsf.predict(data_vect_val)
    acc = accuracy_score(pred, target_val)
    accuracy_mean += acc
    print("Accuracy score is %.4f." % acc)
    
accuracy_mean /= 3
print("\nMean accuracy on 3 folds is %.4f." % accuracy_mean)

Accuracy score is 0.2752.
Accuracy score is 0.2636.
Accuracy score is 0.2766.

Mean accuracy on 3 folds is 0.2718.


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [28]:
tfidf = TfidfVectorizer(max_features = 10000)
data_tfidf = tfidf.fit_transform(data)

In [29]:
# теперь попробуем совместить косинусную метрику и tfidf

kf = KFold(n_splits=3)
mclsf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute', metric='cosine')
accuracy_mean = 0

for train_index, val_index in kf.split(data_tfidf):
    data_tfidf_train, data_tfidf_val = data_tfidf[train_index], data_tfidf[val_index]
    target_train, target_val = target[train_index], target[val_index]
    mclsf.fit(data_tfidf_train, target_train)
    pred = mclsf.predict(data_tfidf_val)
    acc = accuracy_score(pred, target_val)
    accuracy_mean += acc
    print("Accuracy score is %.4f." % acc)
    
accuracy_mean /= 3
print("\nMean accuracy on 3 folds is %.4f." % accuracy_mean)

Accuracy score is 0.5427.
Accuracy score is 0.5338.
Accuracy score is 0.5444.

Mean accuracy on 3 folds is 0.5403.


Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [30]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

In [31]:
test_data = newsgroups['data']
test_target = newsgroups['target']
newgroups_tfidf = tfidf.transform(test_data)

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.

In [32]:
test_predict = mclsf.predict(newgroups_tfidf)
accuracy_score(test_predict, test_target)

0.4629580456718003

Качество отличается в худшую сторону. Возможная причина - наличие слов, которые отсутствовали в train датасете.